In [22]:
import json
from openai import OpenAI
import os
from dotenv import load_dotenv
from pprint import pprint

In [9]:
# Replace 'your_file.json' with the path to your file
file_path = 'your_file.json'

dev_file = 'QuALITY.v1.0.1/QuALITY.v1.0.1.htmlstripped.dev'
test_file = 'QuALITY.v1.0.1/QuALITY.v1.0.1.htmlstripped.test'
train_file = 'QuALITY.v1.0.1/QuALITY.v1.0.1.htmlstripped.train'

def read_data(file):
    data = []
    with open(file, 'r', encoding='utf8') as file:
        for line in file:
            data.append(json.loads(line))
    return data

dev_data = read_data(dev_file)
train_data = read_data(train_file)

In [10]:
sum([len(article['questions']) for article in dev_data[:11]]), sum([len(article['questions']) for article in train_data])

(99, 2523)

In [38]:
load_dotenv()
client = OpenAI(api_key=os.getenv("API_KEY"))

def call_openai(prompt, max_tokens=10, system_prompt=''):
    history = []
    if system_prompt:
        history.append({"role": "system", "content": system_prompt})
    history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=history,
        max_tokens=max_tokens,
    )

    return response.choices[0].message.content

In [49]:
def parse_response(response : str) -> int:
    for c in response:
        if c.isdigit():
            return int(c)

    return 0

In [50]:
def format_prompt(article_text, question, question_options):
    """
    Formats the article text, question, and question options into a prompt.

    Args:
    - article_text (str): The text of the article.
    - question (str): The question to be answered.
    - question_options (list): A list of answer options for the question.

    Returns:
    - str: A formatted prompt string.
    """
    # Formatting the article text
    formatted_article = f"Article:\n{article_text}\n\n"

    # Formatting the question
    formatted_question = f"Question:\n{question}\n\n"

    # Formatting the options
    formatted_options = "Options:\n"
    for idx, option in enumerate(question_options, start=1):
        formatted_options += f"{idx}. {option}\n"

    # Combining all parts into one prompt
    prompt = formatted_article + formatted_question + formatted_options
    return prompt

In [73]:
results = []
for i, row in enumerate(dev_data[:11]):
    # Extracting the article text
    article_text = row["article"]

    # Extracting the questions and their options
    questions = row["questions"]
    for j, question_data in enumerate(questions):
        question = question_data["question"]
        question_options = question_data["options"]
        expected_option_idx = question_data["gold_label"]

        # Formatting the prompt
        formatted_prompt = format_prompt(article_text, question, question_options)
        response = call_openai(formatted_prompt)

        results.append(
            {
                "article_id": row["article_id"],
                "question_id": question_data["question_unique_id"],
                'response_text': response,
                "response": parse_response(response),
                "correct_answer": expected_option_idx,
                "difficulty": question_data["difficult"],
            }
        )

In [74]:
with open('results_first_11.json', 'w') as file:
    json.dump(results, file)

In [19]:
# Read the results from the JSON file
with open('results_first_11.json', 'r') as file:
    results = json.load(file)

# results = [r for r in results if r['difficulty'] == 0]
correct_count = 0
for item in results:
    if item['response'] == item['correct_answer']:
        correct_count += 1

# Calculate accuracy
accuracy = correct_count / len(results) * 100
accuracy


90.9090909090909

In [15]:
len(results)

88

In [20]:
set([r['response'] for r in results])

{0, 1, 2, 3, 4}

In [21]:
[r for r in results if r['response'] == 0]

[{'article_id': '52995',
  'question_id': '52995_I3M5VUMM_1',
  'response_text': 'Answer: \nAs one of two remaining spacemen',
  'response': 0,
  'correct_answer': 3,
  'difficulty': 0},
 {'article_id': '63477',
  'question_id': '63477_65UJ979R_4',
  'response_text': 'The correct answer is: Madame President did not expect',
  'response': 0,
  'correct_answer': 2,
  'difficulty': 0},
 {'article_id': '52995',
  'question_id': '52995_X9XZD7EN_4',
  'response_text': 'Answer:\nThe fear of contracting space cafard.',
  'response': 0,
  'correct_answer': 2,
  'difficulty': 0}]

In [24]:
dev_data[0]['questions'][0].keys()

dict_keys(['question', 'question_unique_id', 'options', 'writer_label', 'gold_label', 'validation', 'speed_validation', 'difficult'])

In [25]:
dev_data[0].keys()

dict_keys(['article_id', 'set_unique_id', 'batch_num', 'writer_id', 'source', 'title', 'year', 'author', 'topic', 'article', 'questions', 'url', 'license'])

In [30]:
# Get all of the question texts in dev_data
q = [question['question'] for article in dev_data[:2] for question in article['questions']]
o = [question['options'] for article in dev_data[:2] for question in article['questions']]

In [35]:
for question, options in zip(q, o):
    print(question)
    print()
    print('\n'.join(extract_topics(question, options)))
    print('----------------')

How much time has passed between Blake's night with Eldoria and his search for Sabrina York in his mind-world?



- Details of Blake's night with Eldoria.
- Time measurements in the mind-world compared to the real world.
- The relationship and timeline between Blake and Sabrina York.
- Any significant events that occurred between Blake's night with Eldoria and his search for Sabrina York.
- The pace at which time passes in Blake's mind-world.
----------------
Why does Deirdre get so upset when Blake Past suggests she go to prom with the young man?

- What is the relationship between Deirdre and Blake Past?
- Does Deirdre have romantic feelings for Blake Past?
- What is Deirdre's reaction to Blake Past's suggestion?
- What is the context of Deirdre's relationship with the young man?
- What is Deirdre's family situation, specifically concerning her parents?
----------------
Why does shame flame in Blake's cheeks when Deirdre goes to prepare Eldoria's dias?

- What is the relationship between Blake, Eldoria, and Deirdre?
- How does Blake perceive Deirdre and Eldoria?
- What is the situation in which 

In [54]:
def extract_topics(question, options):
    prompt = f"Question: {question}\n"
    for i, option in enumerate(options):
        prompt += f"{i+1}: {option}\n\n"

    prompt += "You are a highly capable LLM agent tasked with answering this question. This question can be answered using a document that will be analyzed by one or more sub-agents. Provide a bulleted list of up to 5 relevant sub-questions or topics to keep track of that will help your sub-agents provide the most useful information to you. For example, you could identify relevant 'what' or 'how' questions, name topics to take notes about, or identify relationships to keep track of.\n"
    prompt += "Do not respond with anything other than the sub-questions or topic themselves. List the topics using bullet points and include nothing else."

    response = call_openai(prompt, max_tokens=100)
    topics = response.split("\n")
    return topics

def get_subagent_response(subquestion, article_text):
    paragraphs = article_text.split("\n\n")
    print(f'Article split into {len(paragraphs)} paragraphs.')
    scratchpad = ''

    for paragraph in paragraphs:
        prompt = construct_summary_prompt(paragraph, subquestion, scratchpad)
        paragraph_note = call_openai(prompt, max_tokens=200)

        if 'N/A' not in paragraph_note.upper():
            scratchpad += f'\n- {paragraph_note}'
            print(scratchpad)
            print("-"*20)

    return scratchpad

def construct_summary_prompt(paragraph, subquestion, scratchpad=''):
    subprompt = 'Notes:\n{scratchpad}\n\n' if scratchpad else ''
    # subprompt += "Notes:\n"
    # for i, note in enumerate(notes):
    #     subprompt += f"{i}: {note}\n"
    # subprompt += "\n"
    subprompt += f"Given the following paragraph, record a brief note including any information that may be relevant to the following question: {subquestion}\n"
    subprompt += "If there is no information relevant to the question at all, respond with 'N/A' and nothing else.\n"
    subprompt += f"\nParagraph: {paragraph}\n"
    return subprompt

def construct_synthesis_prompt(subquestions, question, options):
    for subquestion, note in subquestions.items():
        prompt += f"{subquestion}: {note}\n"

    prompt += "\n"
    prompt += f"Given these answered subquestions, {question}\n"

    for i, option in enumerate(options):
        prompt += f"{i+1}: {option}\n"

    return prompt


def call_agent(article_text, question, options):
    subquestions = extract_topics(question, options)
    print(subquestions)

    subquestions_responses = {}
    for subquestion in subquestions:
        print(f"Answering subquestion: {subquestion} ~~~~~~~~~~~~~")
        subagent_response = get_subagent_response(subquestion, article_text)
        subquestions_responses[subquestion] = subagent_response

    print(subquestions_responses)

    prompt = construct_synthesis_prompt(subquestions_responses, question, options)
    response = call_openai(prompt, max_tokens=100)
    return response

In [55]:
article = dev_data[0]['article']
questions = dev_data[0]['questions']
question = questions[0]
question_text = question['question']
options = question['options']

call_agent(article, question_text, options)

['- Who is Blake and what is the significance of his night with Eldoria?', "- What is the context of Blake's search for Sabrina York in his mind-world?", '- What events or markers indicate the passage of time in the narrative?', "- Are there specific dates or time references that relate to Blake's experiences with Eldoria and Sabrina York?", '- Is the time frame mentioned directly in the text or do we need to infer it from the provided information?']
Answering subquestion: - Who is Blake and what is the significance of his night with Eldoria? ~~~~~~~~~~~~~
Answering subquestion: - What is the context of Blake's search for Sabrina York in his mind-world? ~~~~~~~~~~~~~
Answering subquestion: - What events or markers indicate the passage of time in the narrative? ~~~~~~~~~~~~~
Answering subquestion: - Are there specific dates or time references that relate to Blake's experiences with Eldoria and Sabrina York? ~~~~~~~~~~~~~
Answering subquestion: - Is the time frame mentioned directly in t

KeyboardInterrupt: 